In [ ]:
!pip install py3-validate-email
!pip install py3dns

In [2]:
import gspread
import random
import math
import smtplib
import numpy as np
import pandas as pd
from collections import defaultdict
from google.colab import auth
from oauth2client.service_account import ServiceAccountCredentials
from gspread.exceptions import APIError
from gspread.models import Spreadsheet
from google.auth import default
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [3]:
# Autenticação com a API do Google
auth.authenticate_user()
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds, _ = default()
client = gspread.authorize(creds)

In [4]:
# Obtenção da planilha com as respostas
sheet = client.open('Cópia Teste (respostas)').sheet1

# Obtenção dos nomes dos participantes
participantes = [{'Nome': r['Nome Completo'], 'Email': r['E-mail'], 'Telefone': r['Telefone'], 'Grupo': r['Grupo']} for r in sheet.get_all_records()]

In [5]:
nomes_predefinidos = [
    {'Nome': 'Naara', 'Email': 'naaraveira@gmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Pedro', 'Email': 'pedropaulocavalcanten@gmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Gyo', 'Email': 'gyovannamenezes@hotmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Ibsa', 'Email': 'ibsatauanecn@gmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Duda', 'Email': 'mariaeduarda.corcino1@gmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Nathan', 'Email': 'nathanueslinathansouza@hotmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Nataly', 'Email': 'nathalihelendesouza@gmail.com', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Gustavo', 'Email': '', 'Telefone': '', 'Grupo': 'Azul'},
    {'Nome': 'Peter', 'Email': 'jianpiter@gmail.com', 'Telefone': '', 'Grupo': 'Azul'},

    {'Nome': 'Caio Castro', 'Email': 'caio_castro7@hotmail.com', 'Telefone': '', 'Grupo': 'Verde'},
    {'Nome': 'Joel', 'Email': 'joelramosfilhor00@gmail.com', 'Telefone': '', 'Grupo': 'Verde'},
    {'Nome': 'Raissa', 'Email': 'raissamaciels01@gmail.com', 'Telefone': '', 'Grupo': 'Verde'},
    {'Nome': 'Davi', 'Email': 'djordao03@gmail.com', 'Telefone': '', 'Grupo': 'Verde'},
    {'Nome': 'Duda', 'Email': 'eduardapessoa18@gmail.com', 'Telefone': '', 'Grupo': 'Verde'},
    {'Nome': 'Emanuel', 'Email': 'emanuelfelipe.sfarias@gmail.com', 'Telefone': '', 'Grupo': 'Verde'},
    {'Nome': 'Dayana', 'Email': 'dayanans@gmail.com', 'Telefone': '', 'Grupo': 'Verde'},

    {'Nome': 'Rayandson', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Esposa de Rayandson', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Ruthyerre', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Valter', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Leticia', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Livia', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Raithyelle', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Caio', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Paulo', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Bruna', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Aninha', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Felipe', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Dona Ana', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
    {'Nome': 'Rayla', 'Email': '', 'Telefone': '', 'Grupo': 'Vermelho'},
]

In [6]:
# Separa os participantes predefinidos em seus respectivos grupos
grupos = {grupo: [] for grupo in set(participante['Grupo'] for participante in nomes_predefinidos)}
for participante in nomes_predefinidos:
    grupos[participante['Grupo']].append(participante)

In [7]:
# Calcula a quantidade de participantes que cada grupo já tem
for grupo, participantes_grupo in grupos.items():
    quantidade = len(participantes_grupo)
    print(f'O grupo {grupo} já tem {quantidade} participante(s) predefinido(s).')

O grupo Vermelho já tem 14 participante(s) predefinido(s).
O grupo Azul já tem 9 participante(s) predefinido(s).
O grupo Verde já tem 7 participante(s) predefinido(s).


In [8]:
# Calcula o número total de participantes e a quantidade ideal de participantes por grupo
quantidade_total = len(participantes)
grupos_disponiveis = len(set(grupo for grupo in participante['Grupo'] for participante in participantes)) - len(grupos)
quantidade_por_grupo = quantidade_total // grupos_disponiveis
if quantidade_total % grupos_disponiveis != 0:
    quantidade_por_grupo += 1

In [9]:
# Sorteia os participantes não predefinidos e os adiciona ao grupo com menos participantes
participantes_nao_predefinidos = [participante for participante in participantes if participante not in nomes_predefinidos]
random.shuffle(participantes_nao_predefinidos)

for participante in participantes_nao_predefinidos:
    grupo_menos_participantes = min(grupos, key=lambda grupo: len(grupos[grupo]))
    grupos[grupo_menos_participantes].append(participante)
    participante['Grupo'] = grupo_menos_participantes

In [ ]:
for grupo, participantes_grupo in grupos.items():
    quantidade = len(participantes_grupo)
    print('# Grupo ', grupo)
    for participante in participantes_grupo:
        print(participante['Nome'])
    print()

In [ ]:
resultado_sheet = client.create('Teste Gincana da Rede Plenus')

# Adicionar o resultado do sorteio à nova planilha
for i, (grupo, participantes_grupo) in enumerate(grupos.items()):
    grupo_sheet = resultado_sheet.add_worksheet(title=grupo, rows=len(participantes_grupo)+1, cols=len(participantes_grupo[0])-1)
    headers = list(participantes_grupo[0].keys())
    headers.remove('Grupo')
    print(f'Adicionando os grupos na planilha: ', grupo_sheet)
    grupo_sheet.insert_row(headers, 1)
    for j, participante in enumerate(participantes_grupo):
        values = list(participante.values())
        del values[-1]  # remove o valor do grupo da lista de valores
        grupo_sheet.insert_row(values, j+2)

padrao_sheet = resultado_sheet.worksheet('Página1')
resultado_sheet.del_worksheet(padrao_sheet)

In [ ]:
import DNS
from validate_email import validate_email

link_azul = "https://chat.whatsapp.com/ER0BHwVZQjwLlsRwMSbpdH"
link_verde = "https://chat.whatsapp.com/KqQfqr5kfVJ182BZL5LGPq"
link_vermelho = "https://chat.whatsapp.com/K3ZuFWg506EEIEUNTTSwBw"

# Envio do e-mail para cada participante informando a qual equipe pertence
for grupo, participantes in grupos.items():
    for participante in participantes:
        nome = participante["Nome"]
        email = participante["Email"]
        
        if grupo == 'Azul':
            link = link_azul
        elif grupo == 'Verde':
            link = link_verde
        else:
            link = link_vermelho

        mensagem = f"Olá {nome},\n\nVocê foi sorteado para o Grupo {grupo}.\n\nO link para o grupo da sua equipe é:\n{link}\n\nAtenciosamente,\nRede Plenus"
        
        # Configuração da mensagem
        msg = MIMEMultipart()
        msg['From'] = 'ralfferreira995@gmail.com'
        msg['To'] = email
        msg['Subject'] = "Resultado do sorteio da Gincana da Rede Plenus"
        msg.attach(MIMEText(mensagem, 'plain'))

        is_valid = validate_email(
            email
        )
        
        if is_valid:
            # Conexão com o servidor de e-mail e envio da mensagem
            servidor_smtp = smtplib.SMTP('smtp.gmail.com', 587)
            servidor_smtp.starttls()
            servidor_smtp.login('ralfferreira995@gmail.com', 'dgbqzvwnobznhxsu')
            print(f"Enviando e-mail para: {email}")
            servidor_smtp.sendmail('ralfferreira995@gmail.com', email, msg.as_string().encode('utf-8'))
            servidor_smtp.quit()
        else:
          print(f'E-mail inválido: {email}')